<a href="https://colab.research.google.com/github/mayankcircle/Tensorflow-2.0/blob/main/Tensorflow_Tutorial_4_(CNN_using_Sequential_and_Functional_API).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

In [2]:
tf.__version__

'2.3.0'

# Load CIFAR10 Dataset

In [3]:
(X_train, y_train),(X_test,y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [4]:
X_train.shape

(50000, 32, 32, 3)

In [5]:
set(y_train.ravel())

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

**CIFAR10 train dataset has 50000 RGB images of 32 x 32 px of 10 classes.**

We convert each value to flaot32 dtype and normalize for fast training.

In [6]:
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# Basic CNN Model using Sequential API

In [7]:
model = keras.Sequential(
    [
     keras.Input(shape=(32,32,3)), # h,w,channels (Shape of 1 image). By default, data_format is "channels_last"
     # 1st convolutional layer : we use 32 filters of size 3 x 3 with No padding (valid)
     layers.Conv2D(filters=32,kernel_size=3,padding="valid",activation="relu",name="1st_CONV"),
     # max pool layer
     layers.MaxPool2D(pool_size=(2,2),name="1st_POOL"),
     # 2nd convolutional layer : we use 64 filters of size 3 x 3 with No padding (valid)
     # By default, padding is "valid"
     layers.Conv2D(filters=64,kernel_size=3,activation="relu",name="2nd_CONV"),
     # max pool layer
     # By default pool_size = (2,2)
     layers.MaxPool2D(name="2nd_POOL"),
     # 3rd convolutional layer : we use 128 filters of size 3 x 3 with No padding (valid)
     layers.Conv2D(filters=128,kernel_size=3,activation="relu", name="3rd_CONV"),
     # flatten layer
     layers.Flatten(),

     # 1st FC layer
     layers.Dense(64,name="FC1"),
     # Output layer # 10 classes : logits
     layers.Dense(10,name="FC2")

    ]
)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1st_CONV (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
1st_POOL (MaxPooling2D)      (None, 15, 15, 32)        0         
_________________________________________________________________
2nd_CONV (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
2nd_POOL (MaxPooling2D)      (None, 6, 6, 64)          0         
_________________________________________________________________
3rd_CONV (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
FC1 (Dense)                  (None, 64)                1

In [9]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"]
)

In [12]:
# train
model.fit(X_train,y_train,batch_size=64,epochs=2)

Epoch 1/2
782/782 [==============================] - 64s 82ms/step - loss: 0.9790 - accuracy: 0.6581
Epoch 2/2
782/782 [==============================] - 69s 89ms/step - loss: 0.9107 - accuracy: 0.6870


In [13]:
model.evaluate(X_test,y_test,batch_size=64)

157/157 [==============================] - 4s 24ms/step - loss: 0.9711 - accuracy: 0.6625


[0.9711197018623352, 0.6625000238418579]

# Basic CNN model using Functional API

In [28]:
def my_model():
    inputs = keras.Input(shape=(32,32,3))
    x = layers.Conv2D(filters=32,kernel_size=3,name="1st_CONV")(inputs)
    x = layers.BatchNormalization()(x) # we should not applying activation function before Batch Normalization, we need CONV layer RAW Output directly here
    x = keras.activations.relu(x) # apply activation after Batch Norm
    x = layers.MaxPool2D()(x) # by default, pooling window is 2 x 2
    x = layers.Conv2D(filters=64,kernel_size=5,padding="same",name="2nd_CONV")(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Conv2D(filters=128,kernel_size=3,name="3rd_CONV")(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64,activation="relu")(x)
    outputs = layers.Dense(10)(x)
    
    # create model
    model = keras.Model(inputs=inputs, outputs = outputs)

    return model


In [29]:
model = my_model()

In [30]:
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
1st_CONV (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_18 (Batc (None, 30, 30, 32)        128       
_________________________________________________________________
tf_op_layer_Relu_14 (TensorF [(None, 30, 30, 32)]      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
2nd_CONV (Conv2D)            (None, 15, 15, 64)        51264     
_________________________________________________________________
batch_normalization_19 (Batc (None, 15, 15, 64)       

In [31]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"]
)

In [33]:
# train
model.fit(X_train,y_train,batch_size=64,epochs=2)

Epoch 1/2
782/782 [==============================] - 256s 327ms/step - loss: 1.3059 - accuracy: 0.5326
Epoch 2/2
782/782 [==============================] - 252s 322ms/step - loss: 0.9269 - accuracy: 0.6750


In [34]:
model.evaluate(X_test,y_test,batch_size=64)

157/157 [==============================] - 13s 80ms/step - loss: 0.9425 - accuracy: 0.6743


[0.9424744844436646, 0.6743000149726868]